# Liquid Neural Network Basic Usage

This notebook demonstrates the basic usage of the Liquid Neural Network package.

In [ ]:
import torch
import numpy as np
from liquid_neural_network import LiquidS4Model, CfCModel
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Creating a Simple Model

Let's create a basic Liquid S4 model for sequence prediction.

In [ ]:
# Model parameters
input_size = 10
hidden_size = 64
output_size = 10

# Create model
model = LiquidS4Model(input_size, hidden_size, output_size)
print(model)

## 2. Generating Synthetic Data

We'll create some synthetic sequential data for demonstration.

In [ ]:
def generate_sequence_data(batch_size, seq_length, input_size):
    # Generate sinusoidal data
    t = torch.linspace(0, 4*np.pi, seq_length).unsqueeze(0).unsqueeze(-1)
    frequencies = torch.randn(batch_size, 1, 1) * 0.5 + 1.0
    
    # Create input sequences
    x = torch.cat([
        torch.sin(frequencies * t),
        torch.sin(2 * frequencies * t),
        torch.cos(frequencies * t)
    ], dim=-1)
    
    if input_size > x.size(-1):
        x = torch.cat([x] * (input_size // x.size(-1) + 1), dim=-1)
        x = x[:, :, :input_size]
    
    # Target is next-step prediction
    y = torch.roll(x, shifts=-1, dims=1)
    y[:, -1, :] = 0
    
    return x, y

# Generate data
batch_size = 32
seq_length = 100
x, y = generate_sequence_data(batch_size, seq_length, input_size)

# Plot example sequence
plt.figure(figsize=(12, 4))
plt.plot(x[0, :, 0].detach().numpy())
plt.title('Example Input Sequence')
plt.show()

## 3. Training the Model

Now let's train our model on the synthetic data.

In [ ]:
# Training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
n_epochs = 100

# Training loop
losses = []
for epoch in range(n_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(x)
    loss = criterion(output, y)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.show()

## 4. Model Evaluation

Let's evaluate our trained model.

In [ ]:
# Generate test data
x_test, y_test = generate_sequence_data(1, seq_length, input_size)

# Get predictions
model.eval()
with torch.no_grad():
    y_pred = model(x_test)

# Plot results
plt.figure(figsize=(12, 4))
plt.plot(x_test[0, :, 0].numpy(), label='Input')
plt.plot(y_test[0, :, 0].numpy(), label='Target')
plt.plot(y_pred[0, :, 0].numpy(), label='Prediction')
plt.title('Model Predictions vs Target')
plt.legend()
plt.show()

## 5. Using the CfC Model

Let's also try the Closed-form Continuous-time (CfC) model.

In [ ]:
# Create and train CfC model
cfc_model = CfCModel(input_size, hidden_size, output_size)
optimizer = torch.optim.Adam(cfc_model.parameters(), lr=0.001)

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = cfc_model(x)
    loss = criterion(output, y)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Evaluate CfC model
cfc_model.eval()
with torch.no_grad():
    y_pred_cfc, _ = cfc_model(x_test)

# Plot results
plt.figure(figsize=(12, 4))
plt.plot(x_test[0, :, 0].numpy(), label='Input')
plt.plot(y_test[0, :, 0].numpy(), label='Target')
plt.plot(y_pred_cfc[0, :, 0].numpy(), label='CfC Prediction')
plt.title('CfC Model Predictions vs Target')
plt.legend()
plt.show()